## Biomassi mudeldamine masinõppega

Selle skriptiga saad Google Earth Engine'ist saadud tulemuste ja välitööde faili põhjal luua Random Forest masinõppe mudeli. Töö lihtsustamiseks on csv failid juba pilve laetud ning skript loeb need ise pilvest sisse. Skripti kasutamiseks mine hiirega esimesele kastile ja vajuta "Run" nuppu, et koodiplokk läbi jooksutada, tee sama ka järgmiste plokkidega. Kui midagi valesti läheb, saad lihtsa vaevaga uuesti algusest alustada.

### Andmete sisse lugemine ja ettevalmistamine

In [ ]:
# laeme alla vajalikud paketid
import seaborn as sns
import numpy as np
import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load
from sklearn.metrics import r2_score
import shap
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.metrics import mean_squared_error
from IPython.display import FileLink

In [ ]:
# loeme sisse kaugseire andmetega faili, fail on juba eelnevalt GitHubi laetud, eraldi seda laadima ei pea
fp = "data/biomass_2019_results.csv"
obs_data = pd.read_csv(fp)
# kontrollime andmestikku, vaadates esimest viit rida
obs_data.head(5)

In [ ]:
# loeme sisse välitööde csv faili, fail on juba eelnevalt GitHubi laetud, eraldi seda laadima ei pea
fp = "data/fieldwork_data.csv"
fieldwork = pd.read_csv(fp)
# kontrollime andmestikku, vaadates esimest viit rida
fieldwork.head(5)

In [ ]:
# liidame kihid kokku uurimisala ID alusel
data = obs_data.merge(fieldwork, on="ID", how="left", suffixes=('', '_fw'))

In [ ]:
# kontrollime, et kõik uurimisalad esineksid andmestikus ühe korra
if data["ID"].is_unique:
    print("Kõik ID-d on unikaalsed.")
else:
    print("Mõni ID esineb rohkem kui üks kord.")

In [ ]:
# puhastame andmestikku, jättes alles vaid vajalikud veerud
data = data[["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B11", "B12", "ID", "Biomass_tha", "Biomass_womoss_tha", "VH", "VV", "VV/VH", "BSI", "NDVI", "Restoration", "Subsite", "X", "Y", "Tree_cover_10m_radius", "Shrub_cover_10m_radius", "Herb_layer_height_1x1m_cm", "Herb_cover_1x1m"]]
data.columns

In [ ]:
# kontrollime, kuidas jagunevad uurimisala tüübid
data["Subsite"].value_counts()

In [ ]:
# eraldame vaid numbrilised veerud korrelatsioonimaatriksi arvutamiseks
data_num = data.drop(["ID", "Subsite", "Restoration"], axis=1)
# kontrollime, et kõik veerud oleksid float või int
data_num.dtypes

In [ ]:
# kuvame korrelatsioonimaatriksi
correlation_matrix = data_num.corr()

plt.figure(figsize=(10, 8))

sns.heatmap(correlation_matrix, cmap='coolwarm', fmt='.2f', square=True, 
            cbar_kws={"shrink": .8}, linewidths=0.5)

plt.title('Korrelatsioonimaatriks')
plt.xticks(rotation=90)
plt.yticks(rotation=0)   

plt.tight_layout()
plt.show()

In [ ]:
# loome uuritava väärtuse histogrammi
feature = 'Biomass_womoss_tha'

plt.figure(figsize=(6, 4))
data[feature].hist(bins=30, color='skyblue', edgecolor='black')
plt.title(f'Histogram of {feature}')
plt.xlabel(feature)
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
# eraldame andmestikust mitte-metsastunud uurimisalad. metsastunud uurimisaladel on väga vähe rohtset biomassi ning seda on satelliitidel puude alt raske tuvastada
cleaned = data[data["Subsite"] != "Afforested"]
# loeme allesjäänud uurimisalad kokku
len(cleaned)

### Masinõppe mudel
Järgnevalt jagame andmed treening- ja testimisandmeteks ning loome Random Forest regressiooni mudeli. Tavapäraselt jäetakse 70% andmetest mudeli treenimiseks ja 30% testimiseks. Võid proovida ka 80/20 jagamist.
Mudeldatavaks väärtuseks on rohtne biomass ehk Biomass_womoss_tha.

In [ ]:
# eraldame mudeli sisendid (X) ja ennustatava väärtuse (Y)
# müra vältimiseks valime mudelisse vegetatsiooniga seotud kanalid
X = cleaned[["B6", "B8", "B12", "BSI", "VH", "NDVI"]]
y = cleaned["Biomass_womoss_tha"]

In [ ]:
# jagame andmestiku treening-(70%) ja testandmeteks(30%)

test_size = 0.3
random_state = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = random_state)

len(X_train)

Järgmiseks otsime mudelile parimaid hüperparameetreid, proovime erinevaid kombinatsioone puude arvu (n_estimators), puu maksimaalse sügavuse (max_depth) ja lehe minimaalse vaatluste arvu (min_samples_leaf) vahel. Järjestame kombinatsioonid parima treening R2 alusel.

In [ ]:
# otsime optimaalsemaid hüperparameetreid

results = []

for est in [50, 75, 100]:
    for depth in [3, 5, 7]:
        for leaf in [3, 5, 10]:
                rf = RandomForestRegressor(
                    n_estimators=est,
                    max_depth=depth,
                    min_samples_leaf=leaf,
                    max_features='sqrt',
                    random_state=random_state)
                rf.fit(X_train, y_train)
                train_r2 = r2_score(y_train, rf.predict(X_train))
                test_r2 = r2_score(y_test, rf.predict(X_test))
                
                results.append({
                    'n_estimators': est,
                    'max_depth': depth,
                    'min_samples_leaf': leaf,
                    'train_r2': train_r2,
                    'test_r2': test_r2})

results_df = pd.DataFrame(results)

results_df = results_df.sort_values(by='test_r2', ascending=False)

print(results_df.head(20))

**NB! Sisesta järgmisesse koodiplokki parimad hüperparameetrid eelmisest koodiplokist, 
nt 
n_estimators=50, 
max_depth=7, 
min_samples_leaf=3**

In [ ]:
# sisesta parimad hüperparameetrid
regressor = RandomForestRegressor(
    n_estimators= SIIA PARIM n_estimators,
    max_depth= SIIA PARIM max_depth,
    min_samples_leaf= SIIA PARIM min_samples_leaf,
    max_features='sqrt',
    random_state=random_state)

# treenime mudeli
regressor.fit(X_train, y_train)

In [ ]:
# arvutame treeningandmete R2
r2_training = regressor.score(X_train, y_train)
print(f"Treeningandmete R2: {r2_training:.2f}")

In [ ]:
# ennustame testimisandmetele
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)

# arvutame testimisandmete R2
r2_testing = r2_score(y_test, y_test_pred)
print(f"Testimisandmete R2: {r2_testing:.2f}")

In [ ]:
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Treeningandmete ruutkeskmine viga (RMSE): {rmse_train:.2f}")
print(f"Testimisandmete ruutkeskmine viga (RMSE): {rmse_test:.2f}")

## Mudeli analüüsimine ja tulemuste visualiseerimine
Järgmiseks arvutame mudeli SHAP väärtused. SHAP väärtused näitavad, kui palju iga tunnus panustab mudeli väljundisse. Sh näitavad need ka suunda, näiteks kõrge BSI (bare soil index) puhul on prognoositav biomassi mass väiksem, kõrge NDVI puhul aga suurem.

In [ ]:
# arvutame SHAP väärtused mudeli paremaks mõistmiseks
explainer = shap.TreeExplainer(regressor)
shap_values = explainer.shap_values(X_train)

# visualiseerime SHAP väärtuseid
shap.summary_plot(shap_values=shap_values, features=X_train, feature_names=X_train.columns, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values=shap_values, features=X_train, feature_names=X_train.columns)

Viimaseks visualiseerime ennustatud vs tegelikke biomassi väärtuseid. Lisa joonise pealkirja oma nimi.

In [ ]:
# visualiseerime masinõppe tulemusi
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].scatter(y_train, y_train_pred, color='blue', alpha=0.6)
axes[0].plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], '--k')
axes[0].set_xlabel('Tegelik rohtne biomass (t/ha)')
axes[0].set_ylabel('Ennustatud rohtne biomass (t/ha)')
axes[0].set_title(f'Treenimisandmed\n$R^2$ = {r2_training:.2f}')

axes[1].scatter(y_test, y_test_pred, color='green', alpha=0.6)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--k')
axes[1].set_xlabel('Tegelik rohtne biomass (t/ha)')
axes[1].set_ylabel('Ennustatud rohtne biomass (t/ha)')
axes[1].set_title(f'Testimisandmed\n$R^2$ = {r2_testing:.2f}')

# LISA joonise pealkirja oma nimi
fig.suptitle('Biomassi Random Forest mudel, SINU NIMI', fontsize=14)

plt.tight_layout()

plt.show()

print("Lae joonis alla siit:")
FileLink('2019_biomass_model.jpg')

**Harjutuse lõpp – esita ülesande tulemusena oma nimega joonis ning mõnelauseline kokkuvõte tehtust.**